ARKPLANA BLUR VE GÖRSEL EKLEME


In [2]:
import cv2
import mediapipe as mp
import numpy as np

def live_background_replace(mode=0, background_path=None):
 


    mp_selfie_segmentation = mp.solutions.selfie_segmentation
    cap = cv2.VideoCapture(0)

    # Arka plan görselini yükle (mode 1'de)
    if mode == 1 and background_path:
        bg_image = cv2.imread(background_path)
        if bg_image is None:
            print("❌ Arka plan görseli bulunamadı:", background_path)
            return
    else:
        bg_image = None

    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as selfie_segmentation:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            frame = cv2.flip(frame, 1)  # Aynalı görünüm
            h, w, _ = frame.shape

            # Gerekirse arka plan resmini yeniden boyutlandır
            if mode == 1 and bg_image is not None:
                resized_bg = cv2.resize(bg_image, (w, h))

            # Segmentasyon maskesi al
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = selfie_segmentation.process(rgb_frame)
            mask = results.segmentation_mask
            condition = mask > 0.5  # %50 üzeri kişi olarak kabul edilir

            if mode == 0:
                # Bulanıklaştırılmış arka plan
                blurred = cv2.GaussianBlur(frame, (55, 55), 0)
                output_image = np.where(condition[..., None], frame, blurred)

            elif mode == 1 and bg_image is not None:
                # Arka planı resimle değiştir
                output_image = np.where(condition[..., None], frame, resized_bg)

            else:
                output_image = frame  # Her ihtimale karşı orijinal kareyi göster

            cv2.imshow("Kamera - Arka Plan Uygulama", output_image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


In [4]:
live_background_replace(mode=0)
